In [1]:
import pandas as pd
import seaborn as sb
import pylab as py
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from folium import plugins
from geopy.geocoders import Nominatim
import datetime
import statsmodels.api as sm

In [2]:
df = pd.read_csv('combined.csv')
df.head()

,Yearly Microplastics Emission,Mean BMI (female),AlcAmt,Cig/Adult,MedianAge,FirstChildAge,Reproductive Life Span
0,1400,23.267170,747.230,2408.999965,23.724996,24.740000,35.785938
1,1500,23.372925,784.310,2609.750035,23.907336,25.000000,35.823044
2,1800,23.451368,787.020,2701.000035,24.089675,25.133333,35.860151
3,2000,23.547950,796.570,2536.750017,24.272015,25.366667,35.897257
4,2200,23.648248,796.565,2682.749965,24.454354,25.250000,35.934363


In [4]:
df_female = pd.read_csv("female-pop.csv")
df_female = df_female.groupby("Year").sum()
df_female = df_female.reset_index()
df_female = df_female.loc[(df_female['Year'] >= 1980) & (df_female['Year'] <= 2010)]
df_female = df_female.reset_index()
df_female = df_female.drop(columns = ["index"])
df_female.head()

df_cancer = pd.read_csv('cancer.csv')
df_cancer = df_cancer.loc[(df_cancer['Year'] >= 1980) & (df_cancer['Year'] <= 2010)]
df_cancer['Cancer Rate'] = df_cancer.sum(axis=1) - df_cancer["Year"]
df_cancer = df_cancer.reset_index()
df_cancer = df_cancer.drop(columns = ["index"])
df_cancer = pd.concat([df_cancer['Year'], df_cancer['Cancer Rate']], axis = 1)
df_cancer["Cancer Rate"] = df_cancer["Cancer Rate"]/ df_female["PopFemale"] *100
df_cancer.head()

,Year,Cancer Rate
0,1980,0.069059
1,1981,0.069346
2,1982,0.075905
3,1983,0.090922
4,1984,0.092270


In [7]:
X = df

y = df_cancer['Cancer Rate']
y = y.reset_index().drop(columns = "index")
y = y.astype(float)

# Define the modell
X = sm.add_constant(X)
model = sm.OLS(y, X)

# Fit the model
model_fit = model.fit()

# Print the model summary
print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:            Cancer Rate   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     464.6
Date:                Fri, 01 Apr 2022   Prob (F-statistic):           3.24e-23
Time:                        13:52:05   Log-Likelihood:                 139.45
No. Observations:                  31   AIC:                            -262.9
Df Residuals:                      23   BIC:                            -251.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

/Users/justinaquak/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
